In [4]:
import os, math, warnings
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import rasterio
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.strtree import STRtree

warnings.filterwarnings("ignore")

# ================== CONFIG ==================
CATALOG_CSV       = r"D:/Earthquake_Project/Datasets/Earthquake_datasets.csv"
ACTIVE_FAULTS_MARGINS_SHP = r"D:/Earthquake_Project/Datasets/Active faults/gem_active_faults.shp"
OUT_DIR           = r"D:/Earthquake_Project/pga_outputs2"
GRID_STEP_DEG     = 2.0  # Adjustable grid resolution
MIN_MAG           = 4.5
MAX_DIST_KM       = 200.0  # Adjustable max distance
YEARS_RANGE       = (1950, 2025)
FORECAST_YEARS    = [2030, 2050, 2100]
LAGS              = 3  # Adjustable lag window
SEED              = 42
# Updated GMPE-like proxy coefficients (inspired by Boore-Atkinson 2008, adjusted)
C0, C1, C2, H     = -3.5, 1.0, 1.5, 5.0  # More realistic starting point
ACTIVE_CUTOFF     = 1e-6
# ============================================

os.makedirs(OUT_DIR, exist_ok=True)

def haversine_km(lat1, lon1, lat2, lon2):
    d2r = np.pi / 180.0
    f1, f2 = lat1 * d2r, lat2 * d2r
    dlat = (lat2 - lat1) * d2r
    dlon = (lon2 - lon1) * d2r
    a = np.sin(dlat / 2) ** 2 + np.cos(f1) * np.cos(f2) * np.sin(dlon / 2) ** 2
    return 6371.0 * 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))

def pga_proxy(Mw, R_hyp_km, site_factor=1.0):  # Added site factor for flexibility
    ln_pga = C0 + C1 * Mw - C2 * np.log10(R_hyp_km + H) + np.log10(site_factor)
    return np.exp(ln_pga)

def write_geotiff(path, arr2d, lats, lons, dtype="float32"):
    res_x = lons[1] - lons[0]
    res_y = lats[1] - lats[0]
    transform = from_origin(lons.min() - res_x / 2, lats.max() + res_y / 2, res_x, res_y)
    with rasterio.open(path, "w", driver="GTiff",
                       height=arr2d.shape[0], width=arr2d.shape[1],
                       count=1, dtype=dtype, crs="EPSG:4326",
                       transform=transform, compress="lzw") as dst:
        dst.write(np.flipud(arr2d.astype(dtype)), 1)

def array_to_csv(path, arr2d, lats, lons, colname="pga_g"):
    LAT, LON = np.meshgrid(lats, lons, indexing="ij")
    pd.DataFrame({"lat": LAT.ravel(), "lon": LON.ravel(), colname: arr2d.ravel()}).to_csv(path, index=False)

def evaluate(y_true, y_pred, name):
    y_true = y_true.astype(np.float32); y_pred = y_pred.astype(np.float32)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{name} -> MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.4f}")
    return {"model": name, "MAE": mae, "RMSE": rmse, "R2": r2}

# Load single shapefile with spatial index
faults_margins = gpd.read_file(ACTIVE_FAULTS_MARGINS_SHP)
spatial_index = STRtree(faults_margins.geometry)

def nearest_distance_to_geometry(lat, lon, geometry_gdf, spatial_idx):
    point = gpd.points_from_xy([lon], [lat])[0]
    possible_matches_index = list(spatial_idx.query(point))
    if not possible_matches_index:
        return np.nan
    distances = [point.distance(geom) * 111 for geom in [geometry_gdf.geometry.iloc[i] for i in possible_matches_index]]
    return min(distances) if distances else np.nan

# 1) Load catalog
eq = pd.read_csv(CATALOG_CSV)
eq.columns = [c.strip().lower() for c in eq.columns]
for col in ["time", "latitude", "longitude", "depth", "mag"]:
    if col not in eq.columns: raise ValueError(f"Missing column: {col}")
eq["time"] = pd.to_datetime(eq["time"], errors="coerce")
eq = eq.dropna(subset=["time", "latitude", "longitude", "depth", "mag"])
eq = eq[(eq["time"].dt.year >= YEARS_RANGE[0]) & (eq["time"].dt.year <= YEARS_RANGE[1])]
eq = eq[eq["mag"] >= MIN_MAG].copy()
eq["year"] = eq["time"].dt.year.astype(int)
print(f"Events used: {len(eq):,}")

# Add distance feature to earthquake data
print("Calculating distances for earthquakes...")
eq["dist_to_fault_margin_km"] = eq.apply(lambda row: nearest_distance_to_geometry(row["latitude"], row["longitude"], faults_margins, spatial_index), axis=1)
print("Distance calculation for earthquakes completed.")

# 2) Grid
lats = np.arange(-90 + GRID_STEP_DEG / 2, 90, GRID_STEP_DEG)
lons = np.arange(-180 + GRID_STEP_DEG / 2, 180, GRID_STEP_DEG)
nlat, nlon = len(lats), len(lons)
years = np.arange(YEARS_RANGE[0], YEARS_RANGE[1] + 1, dtype=int)
ny = len(years)
year_to_idx = {y: i for i, y in enumerate(years)}
print(f"Grid: {nlat} x {nlon} cells")

lat_bins = np.floor(lats).astype(int)
lon_bins = np.floor(lons).astype(int)
bin_index = {}
for i, la in enumerate(lat_bins):
    for j, lo in enumerate(lon_bins):
        bin_index.setdefault((la, lo), []).append((i, j))

# 3) Annual max PGA per cell with distance features
pga_year = np.zeros((ny, nlat, nlon), dtype=np.float32)
ddeg = MAX_DIST_KM / 111.0
print("Starting PGA computation...")
for idx, ev in eq.iterrows():
    if idx % 1000 == 0:  # Progress update
        print(f"Processed {idx} events")
    y = int(ev["year"]); iy = year_to_idx[y]
    ev_lat, ev_lon = float(ev["latitude"]), float(ev["longitude"])
    dep = max(0.0, float(ev["depth"])); Mw = float(ev["mag"])
    lat_lo = math.floor(ev_lat - ddeg); lat_hi = math.floor(ev_lat + ddeg)
    lon_lo = math.floor(ev_lon - ddeg); lon_hi = math.floor(ev_lon + ddeg)
    cand = []
    for la in range(lat_lo, lat_hi + 1):
        for lo in range(lon_lo, lon_hi + 1):
            if (la, lo) in bin_index:
                cand.extend(bin_index[(la, lo)])
    if not cand: continue
    ci, cj = zip(*cand)
    ci = np.array(ci); cj = np.array(cj)
    dist = haversine_km(ev_lat, ev_lon, lats[ci], lons[cj])
    mask = dist <= MAX_DIST_KM
    if not np.any(mask): continue
    ci = ci[mask]; cj = cj[mask]
    R_hyp = np.sqrt(dist[mask] ** 2 + dep ** 2)
    # Simple site factor based on distance to fault/margin (adjustable)
    site_factor = np.where(dist[mask] < 50.0, 1.2, 1.0)  # Boost near faults
    pga_v = pga_proxy(Mw, R_hyp, site_factor).astype(np.float32)
    pga_year[iy, ci, cj] = np.maximum(pga_year[iy, ci, cj], pga_v)
print("PGA computation finished.")

# Historical base & active mask
hist_max = np.nanmax(pga_year, axis=0).astype(np.float32)
write_geotiff(os.path.join(OUT_DIR, "PGA_hist_max_1950_2025.tif"), hist_max, lats, lons)
array_to_csv(os.path.join(OUT_DIR, "PGA_hist_max_1950_2025.csv"), hist_max, lats, lons)
print("Historical max PGA files saved.")

active_mask = hist_max > ACTIVE_CUTOFF
print("Active cells:", int(active_mask.sum()), "of", nlat * nlon)

# 4) Long format + lags (active cells only) with distance features
rows = []
LAT, LON = np.meshgrid(lats, lons, indexing="ij")
print("Calculating distances for grid cells...")
dist_fault_margin = np.array([[nearest_distance_to_geometry(lat, lon, faults_margins, spatial_index) for lon in lons] for lat in lats])
print("Distance calculation for grid cells completed.")
for yi, y in enumerate(years):
    arr = pga_year[yi]
    arr = np.where(active_mask, arr, np.nan)
    lat_grid, lon_grid = LAT.ravel(), LON.ravel()
    dist_fault_margin_flat = dist_fault_margin.ravel()
    rows.append(pd.DataFrame({
        "year": y,
        "lat": lat_grid,
        "lon": lon_grid,
        "pga_g": arr.ravel(),
        "dist_to_fault_margin_km": dist_fault_margin_flat
    }))
long_df = pd.concat(rows, ignore_index=True).dropna(subset=["pga_g"])

long_df = long_df.sort_values(["lat", "lon", "year"])
for k in range(1, LAGS + 1):
    long_df[f"pga_lag{k}"] = long_df.groupby(["lat", "lon"])["pga_g"].shift(k)
ml_df = long_df.dropna().reset_index(drop=True)

# 5) Time-aware split
train = ml_df[ml_df["year"] <= 2015].copy()
valid = ml_df[(ml_df["year"] > 2015) & (ml_df["year"] <= 2025)].copy()

FEATURES = [f"pga_lag{k}" for k in range(1, LAGS + 1)] + ["lat", "lon", "year", "dist_to_fault_margin_km"]
TARGET = "pga_g"

# Downcast to float32
X_tr = train[FEATURES].astype(np.float32).values
y_tr = train[TARGET].astype(np.float32).values
X_va = valid[FEATURES].astype(np.float32).values
y_va = valid[TARGET].astype(np.float32).values

metrics_rows = []

# 6) Random Forest
rf = RandomForestRegressor(
    n_estimators=100,  # Reduced for speed
    max_depth=10,      # Reduced to prevent overfitting
    max_features=0.5,
    min_samples_split=5,
    min_samples_leaf=2,
    bootstrap=True,
    max_samples=0.5,
    random_state=SEED,
    n_jobs=1
)
print("Training Random Forest...")
rf.fit(X_tr, y_tr)
print("Random Forest training completed.")
pred_va_rf = rf.predict(X_va).astype(np.float32)
metrics_rows.append(evaluate(y_va, pred_va_rf, "RandomForest"))

# 7) XGBoost
xgb_available = True
try:
    from xgboost import XGBRegressor
    xgb = XGBRegressor(
        n_estimators=200,  # Reduced for speed
        learning_rate=0.1,
        max_depth=6,       # Reduced to prevent overfitting
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        tree_method="hist",
        max_bin=64,
        random_state=SEED,
        objective="reg:squarederror",
        n_jobs=1
    )
    print("Training XGBoost...")
    xgb.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False)
    print("XGBoost training completed.")
    pred_va_xgb = xgb.predict(X_va).astype(np.float32)
    metrics_rows.append(evaluate(y_va, pred_va_xgb, "XGBoost"))
except Exception as e:
    xgb_available = False
    print("XGBoost not available, skipping. Reason:", e)

# 8) LSTM
lstm_available = True
try:
    import tensorflow as tf
    from tensorflow.keras import layers, models

    seqs_X_tr, seqs_y_tr = [], []
    seqs_X_va, seqs_y_va = [], []
    for (lat, lon), grp in long_df.groupby(["lat", "lon"]):
        g = grp.sort_values("year")
        vals = g["pga_g"].values.astype("float32")
        yrs = g["year"].values
        dist_fault_margin = g["dist_to_fault_margin_km"].iloc[0]
        if len(vals) < (LAGS + 1): continue
        for t in range(LAGS, len(vals)):
            seq = vals[t - LAGS:t]
            target = vals[t]
            feat_seq = np.column_stack([seq, [dist_fault_margin] * LAGS])
            if yrs[t] <= 2015:
                seqs_X_tr.append(feat_seq); seqs_y_tr.append(target)
            elif 2016 <= yrs[t] <= 2025:
                seqs_X_va.append(feat_seq); seqs_y_va.append(target)

    if len(seqs_X_tr) and len(seqs_X_va):
        X_tr_seq = np.array(seqs_X_tr, dtype=np.float32)
        y_tr_seq = np.array(seqs_y_tr, dtype=np.float32)
        X_va_seq = np.array(seqs_X_va, dtype=np.float32)
        y_va_seq = np.array(seqs_y_va, dtype=np.float32)

        model = models.Sequential([
            layers.Input(shape=(LAGS, 2)),
            layers.LSTM(32, return_sequences=False),  # Reduced units
            layers.Dropout(0.2),  # Added to prevent overfitting
            layers.Dense(1)
        ])
        model.compile(optimizer="adam", loss="mae")
        print("Training LSTM...")
        model.fit(X_tr_seq, y_tr_seq, epochs=10, batch_size=128, validation_split=0.2, verbose=1)
        print("LSTM training completed.")
        lstm_pred_va = model.predict(X_va_seq, verbose=0).ravel().astype(np.float32)
        metrics_rows.append(evaluate(y_va_seq, lstm_pred_va, "LSTM"))

        # Rolling forecast for LSTM
        lstm_state = {}
        for (lat, lon), grp in long_df.groupby(["lat", "lon"]):
            g = grp.sort_values("year")
            vals = g[g["year"] <= 2025]["pga_g"].values.astype("float32")
            dist_fault_margin = g["dist_to_fault_margin_km"].iloc[0]
            if len(vals) >= LAGS:
                lstm_state[(lat, lon)] = list(zip(vals[-LAGS:], [dist_fault_margin] * LAGS))

        lstm_fc_grids = {yr: np.full((len(lats), len(lons)), np.nan, dtype=np.float32) for yr in FORECAST_YEARS}
        for yr in sorted(FORECAST_YEARS):
            for step in range(2026, yr + 1):
                for (lat, lon), tail in lstm_state.items():
                    if len(tail) < LAGS: continue
                    x = np.array([t[0] for t in tail], dtype=np.float32)[None, :, None]
                    yhat = float(model.predict(x, verbose=0)[0, 0])
                    tail.append((yhat, tail[0][1]))
                    if len(tail) > LAGS: tail.pop(0)
            for (lat, lon), tail in lstm_state.items():
                i = np.where(lats == lat)[0]; j = np.where(lons == lon)[0]
                if i.size and j.size:
                    lstm_fc_grids[yr][i[0], j[0]] = tail[-1][0]

        for yr, grid in lstm_fc_grids.items():
            write_geotiff(os.path.join(OUT_DIR, f"PGA_LSTM_{yr}.tif"), grid, lats, lons)
            array_to_csv(os.path.join(OUT_DIR, f"PGA_LSTM_{yr}.csv"), grid, lats, lons)
    else:
        print("Not enough sequences for LSTM; skipping.")
except Exception as e:
    lstm_available = False
    print("LSTM step skipped. Reason:", e)

# 9) Rolling forecast for RF and XGB
def roll_forecast(model, start_year=2025, target_years=FORECAST_YEARS):
    last = long_df[long_df["year"] <= start_year].sort_values(["lat", "lon", "year"])
    buff = last.groupby(["lat", "lon"]).apply(lambda g: list(zip(g["pga_g"].tail(LAGS), g["dist_to_fault_margin_km"].head(1)))).to_dict()
    preds = {}
    for yr in sorted(target_years):
        for y in range(start_year + 1, yr + 1):
            feats_rows, idx_rows = [], []
            for (lat, lon), tail in buff.items():
                if len(tail) < LAGS: continue
                row = {f"pga_lag{k}": tail[-k][0] for k in range(1, LAGS + 1)}
                row.update({"lat": lat, "lon": lon, "year": y, "dist_to_fault_margin_km": tail[0][1]})
                feats_rows.append(row); idx_rows.append((lat, lon))
            if not feats_rows: break
            F = pd.DataFrame(feats_rows)[FEATURES].astype(np.float32).values
            yhat = model.predict(F).astype(np.float32)
            for (lat, lon), val in zip(idx_rows, yhat):
                tail = buff[(lat, lon)]
                tail.append((float(val), tail[0][1]))
                if len(tail) > LAGS: tail.pop(0)
        grid = np.full((len(lats), len(lons)), np.nan, dtype=np.float32)
        for (lat, lon), tail in buff.items():
            i = np.where(lats == lat)[0]; j = np.where(lons == lon)[0]
            if i.size and j.size: grid[i[0], j[0]] = tail[-1][0]
        preds[yr] = grid
    return preds

if xgb_available:
    print("Generating XGBoost forecasts...")
    xgb_fc = roll_forecast(xgb)
    for yr, grid in xgb_fc.items():
        write_geotiff(os.path.join(OUT_DIR, f"PGA_XGB_{yr}.tif"), grid, lats, lons)
        array_to_csv(os.path.join(OUT_DIR, f"PGA_XGB_{yr}.csv"), grid, lats, lons)
    print("XGBoost forecasts completed.")

rf_fc = roll_forecast(rf)
print("Generating Random Forest forecasts...")
for yr, grid in rf_fc.items():
    write_geotiff(os.path.join(OUT_DIR, f"PGA_RF_{yr}.tif"), grid, lats, lons)
    array_to_csv(os.path.join(OUT_DIR, f"PGA_RF_{yr}.csv"), grid, lats, lons)
print("Random Forest forecasts completed.")

# 10) Save metrics
metrics_df = pd.DataFrame(metrics_rows)
metrics_path = os.path.join(OUT_DIR, "model_metrics_validation_2016_2025.csv")
metrics_df.to_csv(metrics_path, index=False)
print("\nSaved metrics to:", metrics_path)
print("Forecast files saved in:", OUT_DIR)
print("Expected outputs include:")
print("  - PGA_RF_{2030,2050,2100}.tif / .csv")
if xgb_available:
    print("  - PGA_XGB_{2030,2050,2100}.tif / .csv")
if lstm_available:
    print("  - PGA_LSTM_{2030,2050,2100}.tif / .csv")

Events used: 284,368
Calculating distances for earthquakes...
Distance calculation for earthquakes completed.
Grid: 90 x 180 cells
Starting PGA computation...
Processed 0 events
Processed 1000 events
Processed 6000 events
Processed 8000 events
Processed 9000 events
Processed 12000 events
Processed 13000 events
Processed 16000 events
Processed 19000 events
Processed 20000 events
Processed 21000 events
Processed 22000 events
Processed 23000 events
Processed 25000 events
Processed 26000 events
Processed 27000 events
Processed 28000 events
Processed 35000 events
Processed 37000 events
Processed 47000 events
Processed 48000 events
Processed 50000 events
Processed 51000 events
Processed 52000 events
Processed 53000 events
Processed 55000 events
Processed 56000 events
Processed 59000 events
Processed 62000 events
Processed 65000 events
Processed 71000 events
Processed 73000 events
Processed 83000 events
Processed 87000 events
Processed 89000 events
Processed 90000 events
Processed 91000 event

In [12]:
"""
Global PGA (Peak Ground Acceleration) Forecasts for 2030, 2050, 2100
====================================================================

Inputs (already uploaded by user):
- /mnt/data/Earthquake.csv               : global earthquake catalogue
- /mnt/data/gem_active_faults.shp (+.*) : global active fault lines (GEM)

Outputs:
- /mnt/data/pga_forecast_2030.tif, _2050.tif, _2100.tif (GeoTIFF rasters in g units)
- /mnt/data/pga_forecast_{year}.csv     (grid cell centroids + PGA)
- /mnt/data/pga_features_{year}.parquet (features used for ML)

Notes
-----
1) This script provides a *complete, reproducible pipeline* you can run locally.
   It includes:
   - Data loading & cleaning
   - Global equal-angle grid generation
   - Spatial features: seismicity rate, b-value, max magnitude, nearest-fault distance, fault density
   - Physics-inspired baseline (Gutenberg–Richter + simple attenuation) to derive a prior PGA
   - ML model (GradientBoostingRegressor) that learns residual corrections
   - Forecasts for horizons H = 5y (to 2030), 25y (to 2050), 75y (to 2100) from a reference year

2) GMPE: By default we use a *simple magnitude–distance attenuation proxy* to turn an
   extreme-event magnitude (per cell & horizon) into a PGA prior. Coefficients are tunable.
   If you have OpenQuake Hazardlib installed, you can plug in a proper GMPE (see the
   TODO marked section), which is recommended for production.

3) Performance: global computations can be heavy. Start with a coarser grid (e.g., 1°) and
   smaller search radius, then refine.

4) Column names: set them in the CONFIG section to match your CSV.
"""

from __future__ import annotations
import os
import math
import warnings
from dataclasses import dataclass
from typing import Tuple, List

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
from shapely.ops import unary_union
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from joblib import Parallel, delayed

# Optional (only used if raster export is desired and rasterio is available)
try:
    import rasterio
    from rasterio.transform import from_origin
except Exception:  # pragma: no cover
    rasterio = None
    from_origin = None

warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------- CONFIG ---------------------------------
@dataclass
class Config:
    EQ_CSV: str = r"D:/Earthquake_Project/Datasets/Earthquake_datasets.csv"
    FAULTS_SHP: str = r"D:/Earthquake_Project/Datasets/Active faults/gem_active_faults.shp"

    # Map your earthquake CSV columns here
    LAT_COL: str = "latitude"   # alternatives: lat, Latitude
    LON_COL: str = "longitude"  # alternatives: lon, Longitude
    MAG_COL: str = "mag"         # magnitude (Mw preferred)
    DEP_COL: str = "depth"       # in km (if missing, set a constant)
    TIME_COL: str = "time"       # datetime string

    # Catalogue filtering
    YEAR_MIN: int = 1970
    YEAR_MAX: int = 2024
    MMIN: float = 4.5             # completeness magnitude threshold

    # Global grid
    GRID_RES_DEG: float = 1.0     # 1° grid to start; later try 0.5° or 0.25°

    # Feature radii (km)
    R_SEIS_KM: float = 500.0      # radius to gather earthquakes for local stats
    R_FAULT_KM: float = 200.0     # radius for fault density
    SEARCH_MAX_KM: float = 300.0  # max distance for event-to-cell PGA calculation (labels)

    # Horizons (years ahead from REF_YEAR)
    REF_YEAR: int = 2025          # today ~ 2025-08; adjust as needed
    HORIZONS: Tuple[int, ...] = (5, 25, 75)  # -> 2030, 2050, 2100

    # Simple attenuation proxy coefficients (log10 PGA in g)
    # log10(PGA_g) = A + B*M - C*log10(Rhyp + R0) - D*Rhyp_km
    A: float = -1.20
    B: float = 0.30
    C: float = 1.10
    D: float = 0.0005
    R0: float = 10.0
    DEP_FLOOR_KM: float = 5.0

    N_JOBS: int = max(1, os.cpu_count() - 1)
    RANDOM_STATE: int = 42

CFG = Config()

# ------------------------- HELPER FUNCTIONS ----------------------------
EARTH_RADIUS_KM = 6371.0088


def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance in km between two points (lat/lon in degrees)."""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return EARTH_RADIUS_KM * c


def rhyp_km(r_epi_km: np.ndarray, depth_km: np.ndarray) -> np.ndarray:
    return np.sqrt(r_epi_km**2 + np.maximum(depth_km, CFG.DEP_FLOOR_KM)**2)


def simple_pga_proxy_g(M: np.ndarray, Rhyp_km: np.ndarray) -> np.ndarray:
    """A tunable, simple magnitude–distance attenuation for PGA (in g).
    This is a *placeholder*. For production, replace with a GMPE from OpenQuake.
    """
    log10_pga = CFG.A + CFG.B*M - CFG.C*np.log10(Rhyp_km + CFG.R0) - CFG.D*Rhyp_km
    return np.maximum(10**log10_pga, 1e-6)  # clamp tiny values


def year_fraction(dt: pd.Series) -> pd.Series:
    return dt.dt.year + (dt.dt.dayofyear - 1)/365.25


def gr_b_value(mags: np.ndarray, mmin: float) -> float:
    """Compute b-value by Aki (1965) MLE; returns np.nan if not enough data."""
    mags = mags[~np.isnan(mags)]
    if mags.size < 20:
        return np.nan
    mean_m = mags.mean()
    return (np.log10(np.e)) / (mean_m - mmin + 1e-6)


def make_global_grid(res_deg: float) -> gpd.GeoDataFrame:
    lats = np.arange(-90 + res_deg/2, 90, res_deg)
    lons = np.arange(-180 + res_deg/2, 180, res_deg)
    rows = []
    for lat in lats:
        for lon in lons:
            cell = box(lon - res_deg/2, lat - res_deg/2, lon + res_deg/2, lat + res_deg/2)
            rows.append({"lat": float(lat), "lon": float(lon), "geometry": cell})
    gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
    return gdf


def points_within_radius_idx(lat0, lon0, lats, lons, radius_km) -> np.ndarray:
    d = haversine_km(lat0, lon0, lats, lons)
    return np.where(d <= radius_km)[0]


def fault_density_km_per_area(cell_poly, faults_gdf, buffer_km=CFG.R_FAULT_KM) -> float:
    # approximate by projecting locally to EPSG:3857 for length
    local = gpd.GeoDataFrame(geometry=[cell_poly], crs="EPSG:4326").to_crs(3857)
    faults_3857 = faults_gdf.to_crs(3857)
    buf = local.buffer(buffer_km*1000).iloc[0]
    sel = faults_3857[faults_3857.intersects(buf)]
    if sel.empty:
        return 0.0
    total_len_m = sel.length.sum()
    area_km2 = local.area.iloc[0] / 1e6
    return float((total_len_m / 1000.0) / max(area_km2, 1e-6))


# --------------------------- DATA LOADING ------------------------------

def load_data(cfg: Config):
    eq = pd.read_csv(cfg.EQ_CSV, low_memory=False)
    # Standardize column names
    cols = {cfg.LAT_COL: "lat", cfg.LON_COL: "lon", cfg.MAG_COL: "mag"}
    if cfg.DEP_COL in eq.columns:
        cols[cfg.DEP_COL] = "depth"
    if cfg.TIME_COL in eq.columns:
        cols[cfg.TIME_COL] = "time"
    eq = eq.rename(columns=cols)

    # Parse time
    if "time" in eq.columns:
        eq["time"] = pd.to_datetime(eq["time"], errors="coerce")
    else:
        eq["time"] = pd.NaT

    # Filter
    if eq["time"].notna().any():
        eq = eq[(eq["time"].dt.year >= cfg.YEAR_MIN) & (eq["time"].dt.year <= cfg.YEAR_MAX)]
    eq = eq[eq["mag"] >= cfg.MMIN].copy()
    eq["depth"] = eq.get("depth", pd.Series(np.full(len(eq), 10.0)))
    eq = eq.dropna(subset=["lat","lon","mag"]).reset_index(drop=True)

    faults = gpd.read_file(cfg.FAULTS_SHP)
    if faults.crs is None:
        faults.set_crs("EPSG:4326", inplace=True)
    else:
        faults = faults.to_crs("EPSG:4326")

    return eq, faults


# ----------------------- FEATURE ENGINEERING --------------------------

def build_features_for_grid(eq: pd.DataFrame, faults: gpd.GeoDataFrame, grid: gpd.GeoDataFrame, cfg: Config) -> pd.DataFrame:
    # Pre-arrays for speed
    eq_lat = eq["lat"].to_numpy()
    eq_lon = eq["lon"].to_numpy()
    eq_mag = eq["mag"].to_numpy()
    eq_dep = eq["depth"].to_numpy()
    eq_year = np.where(eq["time"].notna(), eq["time"].dt.year.to_numpy(), np.full(len(eq), np.nan))

    T_years = np.nanmax(eq_year) - np.nanmin(eq_year) if np.isfinite(eq_year).all() else (cfg.YEAR_MAX - cfg.YEAR_MIN + 1)
    T_years = max(T_years, 1)

    rows = []

    def process_cell(row):
        lat0 = row["lat"]; lon0 = row["lon"]; poly = row["geometry"]
        idx = points_within_radius_idx(lat0, lon0, eq_lat, eq_lon, cfg.R_SEIS_KM)
        n = idx.size
        if n == 0:
            fden = fault_density_km_per_area(poly, faults, cfg.R_FAULT_KM)
            return {
                "lat": lat0, "lon": lon0, "rate_py": 0.0, "b": np.nan, "mmax_hist": np.nan,
                "m95": np.nan, "dist_fault_km": np.nan, "fault_density": fden,
            }
        mags = eq_mag[idx]
        b = gr_b_value(mags, cfg.MMIN)
        rate_py = n / T_years
        mmax_hist = float(np.nanmax(mags)) if mags.size else np.nan
        # 95th percentile magnitude as robust high-tail
        m95 = float(np.nanpercentile(mags, 95)) if mags.size else np.nan

        # Nearest fault distance (km) using centroid
        centroid = gpd.GeoSeries([poly.centroid], crs="EPSG:4326").to_crs(3857)
        faults3857 = faults.to_crs(3857)
        dmin_m = faults3857.distance(centroid.iloc[0]).min() if len(faults3857) else np.nan
        dist_fault_km = float(dmin_m / 1000.0) if np.isfinite(dmin_m) else np.nan

        # Fault density around cell
        fden = fault_density_km_per_area(poly, faults, cfg.R_FAULT_KM)

        return {
            "lat": lat0, "lon": lon0, "rate_py": float(rate_py), "b": float(b) if np.isfinite(b) else np.nan,
            "mmax_hist": mmax_hist, "m95": m95, "dist_fault_km": dist_fault_km, "fault_density": fden,
        }

    rows = Parallel(n_jobs=CFG.N_JOBS, prefer="threads")(delayed(process_cell)(grid.iloc[i]) for i in range(len(grid)))
    feat = pd.DataFrame(rows)
    return feat


# ---------------------- EXTREME MAGNITUDE MODEL -----------------------

def expected_extreme_mag(rate_py: float, b: float, horizon_y: int, mmin: float) -> float:
    """Simple GR-based extreme magnitude in horizon H years.
    If b is missing, fall back to mmin + log10(N) with b=1.
    N = rate_py * H
    Mmax ~ mmin + (1/b)*log10(max(N, 1))
    """
    N = max(rate_py * horizon_y, 1e-6)
    if not np.isfinite(b) or b <= 0:
        b = 1.0
    return mmin + (1.0 / b) * math.log10(max(N, 1.0))


def pga_prior_from_features(feat_row: pd.Series, horizon_y: int, cfg: Config) -> float:
    Mext = max(expected_extreme_mag(feat_row["rate_py"], feat_row["b"], horizon_y, cfg.MMIN), feat_row.get("m95", cfg.MMIN))
    # Use distance to nearest fault as a proxy for expected source distance
    R_epi = max(feat_row.get("dist_fault_km", 50.0), 10.0)
    Rh = math.sqrt(R_epi**2 + cfg.DEP_FLOOR_KM**2)
    pga_g = simple_pga_proxy_g(np.array([Mext]), np.array([Rh]))[0]
    # Scale slightly by fault density (more faults -> higher shaking potential)
    fden = feat_row.get("fault_density", 0.0)
    scale = 1.0 + 0.05 * math.tanh(fden / 10.0)
    return float(pga_g * scale)


# --------------------------- ML CORRECTION -----------------------------

def make_training_labels(eq: pd.DataFrame, grid: gpd.GeoDataFrame, cfg: Config, label_window_y: int = 10) -> pd.DataFrame:
    """Create historical labels: for reference years in the past, compute the *observed* max
    (proxy) PGA in the next `label_window_y` using events within SEARCH_MAX_KM.
    This lets the ML model learn corrections to the prior.
    """
    if "time" not in eq.columns or eq["time"].isna().all():
        raise ValueError("Earthquake CSV lacks a usable time column; cannot create labels.")

    years = np.arange(max(CFG.YEAR_MIN, 1980), CFG.YEAR_MAX - label_window_y, label_window_y)

    records = []
    # Pre arrays
    eq_lat = eq["lat"].to_numpy(); eq_lon = eq["lon"].to_numpy()
    eq_mag = eq["mag"].to_numpy(); eq_dep = eq["depth"].to_numpy()
    eq_year = eq["time"].dt.year.to_numpy()

    for y0 in years:
        y1 = y0 + label_window_y
        mask = (eq_year >= y0) & (eq_year < y1)
        lat_sub = eq_lat[mask]; lon_sub = eq_lon[mask]
        mag_sub = eq_mag[mask]; dep_sub = eq_dep[mask]
        if lat_sub.size == 0:
            continue

        def label_cell(row):
            lat0, lon0 = row["lat"], row["lon"]
            d_epi = haversine_km(lat0, lon0, lat_sub, lon_sub)
            sel = d_epi <= cfg.SEARCH_MAX_KM
            if not np.any(sel):
                return 1e-6
            Rh = rhyp_km(d_epi[sel], dep_sub[sel])
            pga = simple_pga_proxy_g(mag_sub[sel], Rh)
            return float(np.max(pga))

        labels = Parallel(n_jobs=CFG.N_JOBS, prefer="threads")(delayed(label_cell)(grid.iloc[i]) for i in range(len(grid)))
        df = pd.DataFrame({"year0": y0, "year1": y1, "lat": grid["lat"].values, "lon": grid["lon"].values, "pga_obs": labels})
        records.append(df)

    return pd.concat(records, ignore_index=True) if records else pd.DataFrame()


def train_ml_correction(features: pd.DataFrame, labels: pd.DataFrame) -> GradientBoostingRegressor:
    df = features.merge(labels, on=["lat","lon"], how="inner")
    if df.empty:
        raise RuntimeError("No training data could be constructed. Check time ranges and columns.")
    X = df[["rate_py","b","mmax_hist","m95","dist_fault_km","fault_density"]].fillna(0.0)
    y = np.log1p(df["pga_obs"])  # stabilize
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=CFG.RANDOM_STATE)
    model = GradientBoostingRegressor(random_state=CFG.RANDOM_STATE)
    model.fit(Xtr, ytr)
    yhat = model.predict(Xte)
    print("ML correction R2:", r2_score(yte, yhat))
    print("ML correction MAE (log1p):", mean_absolute_error(yte, yhat))
    return model


# --------------------------- FORECASTING -------------------------------

def forecast_pga(features: pd.DataFrame, model: GradientBoostingRegressor | None, horizon_y: int, cfg: Config) -> pd.DataFrame:
    # Prior from physics-inspired model
    prior = features.apply(lambda r: pga_prior_from_features(r, horizon_y, cfg), axis=1)
    out = features[["lat","lon"]].copy()
    out["pga_prior"] = prior.values

    if model is not None:
        X = features[["rate_py","b","mmax_hist","m95","dist_fault_km","fault_density"]].fillna(0.0)
        corr = np.expm1(model.predict(X))  # predicted observed pga scale in training window
        # Blend: geometric mean between prior and correction scale
        out["pga_ml"] = np.sqrt(out["pga_prior"] * np.maximum(corr, 1e-6))
        out["pga_g"] = out["pga_ml"].clip(lower=1e-6)
    else:
        out["pga_g"] = out["pga_prior"].clip(lower=1e-6)

    return out


# ----------------------------- EXPORT ---------------------------------

def export_csv(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)
    print(f"Saved {path}")


def export_geotiff(df: pd.DataFrame, cfg: Config, path: str):
    if rasterio is None:
        print("rasterio not available; skipping GeoTIFF export")
        return
    # Build raster grid
    res = cfg.GRID_RES_DEG
    lats = np.arange(-90 + res/2, 90, res)
    lons = np.arange(-180 + res/2, 180, res)
    arr = np.full((len(lats), len(lons)), np.nan, dtype=np.float32)
    # Map df values to raster
    idx = {(round(r.lat,6), round(r.lon,6)): r.pga_g for r in df.itertuples(index=False)}
    for i, lat in enumerate(lats[::-1]):  # top to bottom
        for j, lon in enumerate(lons):
            v = idx.get((round(lat,6), round(lon,6)), np.nan)
            arr[i, j] = v
    transform = from_origin(-180, 90, res, res)
    with rasterio.open(
        path,
        "w",
        driver="GTiff",
        height=arr.shape[0],
        width=arr.shape[1],
        count=1,
        dtype=arr.dtype,
        crs="EPSG:4326",
        transform=transform,
        compress="lzw",
        nodata=np.nan,
    ) as dst:
        dst.write(arr, 1)
    print(f"Saved {path}")


# ------------------------------ MAIN ----------------------------------

def main():
    cfg = CFG
    print("Loading data ...")
    eq, faults = load_data(cfg)
    print(f"Catalogue: {len(eq):,} events | Faults: {len(faults):,} features")

    print("Building global grid ...")
    grid = make_global_grid(cfg.GRID_RES_DEG)

    print("Computing features (this can take time globally) ...")
    feat = build_features_for_grid(eq, faults, grid, cfg)

    # Historical labels for ML correction (optional but recommended)
    print("Creating historical labels for ML correction ...")
    try:
        labels = make_training_labels(eq, grid, cfg, label_window_y=10)
        if not labels.empty:
            model = train_ml_correction(feat, labels)
        else:
            print("No labels were created; proceeding without ML correction.")
            model = None
    except Exception as e:
        print("Label creation failed:", e)
        model = None

    # Forecast for each horizon
    for H in cfg.HORIZONS:
        year = cfg.REF_YEAR + H
        print(f"Forecasting PGA for horizon {H}y (≈ {year}) ...")
        pred = forecast_pga(feat, model, H, cfg)
        pred.to_parquet(f"/mnt/data/pga_features_{year}.parquet", index=False)
        export_csv(pred[["lat","lon","pga_g"]], f"/mnt/data/pga_forecast_{year}.csv")
        export_geotiff(pred[["lat","lon","pga_g"]], cfg, f"/mnt/data/pga_forecast_{year}.tif")

    print("Done.")


if __name__ == "__main__":
    main()


Loading data ...
Catalogue: 271,987 events | Faults: 16,195 features
Building global grid ...
Computing features (this can take time globally) ...


KeyboardInterrupt: 

In [ ]:
# =========================
# INSTALL NOTES (read only)
# =========================
# If you want the *real* GMPEs, install OpenQuake Engine (which contains hazardlib):
#   conda install -c conda-forge openquake.engine
# or:
#   git clone --depth=1 https://github.com/gem/oq-engine.git
#   cd oq-engine && python install.py devel   # then activate the created env
#
# This script will still run end-to-end without OpenQuake by using a built-in,
# approximate GMPE fallback (clearly marked below).

import os, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point

# ML
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Raster out
import rasterio as rio
from rasterio.transform import from_origin

# ---------------- USER PATHS ----------------
CSV = r"D:/Earthquake_Project/Datasets/Earthquake_datasets.csv"     # catalog csv
FAULTS = r"D:/Earthquake_Project/Datasets/Active faults/gem_active_faults.shp"  # optional, not required to run
OUT_DIR = r"D:/Earthquake_Project"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------- SETTINGS ----------------
MIN_MAG = 4.5         # filter small events
MAX_DIST_KM = 300.0   # consider events within this distance for hazard
GRID_RES_DEG = 0.5    # ~55 km at equator; use 0.25 for finer (slower)
TARGET_POE = 0.10     # 10% in ...
TARGET_YEARS = 50     # ... 50 years
VS30 = 760.0          # "rock" site class
RUP_DEPTH_DEFAULT = 8.0  # fallback rupture depth if missing

# ---------------- 0) TRY OPENQUAKE; ELSE USE FALLBACK ----------------
USE_OPENQUAKE = False
try:
    from openquake.hazardlib.gsim.boore_atkinson_2008 import BooreAtkinson2008
    from openquake.hazardlib.const import StdDev
    from openquake.hazardlib.imt import PGA
    from openquake.hazardlib import contexts
    gsim = BooreAtkinson2008()
    imt = PGA()
    USE_OPENQUAKE = True
    print("Using OpenQuake Boore-Atkinson (2008) GMPE.")
except Exception as e:
    print("OpenQuake not available — using built-in approximate GMPE fallback.")
    USE_OPENQUAKE = False

# ---------------- 1) LOAD & CLEAN CATALOG ----------------
if not os.path.exists(CSV):
    raise FileNotFoundError(f"Catalog not found: {CSV}")

eq = pd.read_csv(CSV)

# normalize column names
rename_map = {
    "latitude":"lat", "Latitude":"lat",
    "longitude":"lon", "Longitude":"lon",
    "depth":"depth_km", "Depth":"depth_km", "depth_km":"depth_km",
    "mag":"mag", "magnitude":"mag", "Magnitude":"mag",
}
for k,v in list(rename_map.items()):
    if k in eq.columns:
        eq = eq.rename(columns={k:v})

# sanity checks
needed = ["lat","lon","mag"]
missing = [c for c in needed if c not in eq.columns]
if missing:
    raise ValueError(f"CSV must contain columns {needed}; missing: {missing}")

eq = eq.dropna(subset=["lat","lon","mag"]).copy()

# parse time
if "time" in eq.columns:
    eq["time"] = pd.to_datetime(eq["time"], errors="coerce", utc=True)
else:
    # fabricate a simple time if missing (assume all events in a 10-yr span)
    eq["time"] = pd.Timestamp("2000-01-01", tz="UTC") + pd.to_timedelta(np.arange(len(eq)) % 3650, unit="D")
eq = eq.dropna(subset=["time"])

# prefer Mw if provided
if "magType" in eq.columns:
    eq["magType"] = eq["magType"].astype(str)
    mw_mask = eq["magType"].str.upper().str.contains("MW", na=False)
    eq["mag_Mw"] = np.where(mw_mask, eq["mag"], np.nan)
else:
    eq["magType"] = np.nan
    eq["mag_Mw"] = np.nan

eq["mag_use"] = eq["mag_Mw"].fillna(eq["mag"])

# filter by magnitude
eq = eq[eq["mag_use"] >= MIN_MAG].copy()
if eq.empty:
    raise ValueError("No events remain after MIN_MAG filter.")

# duration of catalog (years)
years_span = (eq["time"].max() - eq["time"].min()).days / 365.25
years_span = max(1.0, float(years_span))

# ensure depth
if "depth_km" not in eq.columns:
    eq["depth_km"] = RUP_DEPTH_DEFAULT
eq["depth_km"] = eq["depth_km"].fillna(RUP_DEPTH_DEFAULT)

# GeoDataFrame
eq_gdf = gpd.GeoDataFrame(
    eq,
    geometry=gpd.points_from_xy(eq["lon"].astype(float), eq["lat"].astype(float)),
    crs="EPSG:4326"
)

# ---------------- 2) BUILD GLOBAL GRID ----------------
lons = np.arange(-180 + GRID_RES_DEG/2, 180, GRID_RES_DEG)
lats = np.arange(-90 + GRID_RES_DEG/2,   90, GRID_RES_DEG)
grid_ll = [(lo, la) for la in lats for lo in lons]
grid = pd.DataFrame(grid_ll, columns=["lon","lat"])
grid_gdf = gpd.GeoDataFrame(grid, geometry=gpd.points_from_xy(grid["lon"], grid["lat"]), crs="EPSG:4326")

# ---------------- 3) HAVERSINE ----------------
def haversine_km(lon1, lat1, lon2, lat2):
    R = 6371.0
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2.0 * R * np.arcsin(np.sqrt(a))

# ---------------- 4) GMPE WRAPPERS ----------------
def _pga_ln_stats_openquake(M, Rrup_km, vs30=VS30, ztor=None, rake=None):
    """Mean ln(PGA) and TOTAL sigma via OpenQuake BA2008 (rock)."""
    sctx = contexts.SitesContext()
    sctx.vs30 = np.array([vs30], dtype=float)
    sctx.vs30measured = np.array([False])
    sctx.z1pt0 = np.array([np.nan])
    sctx.z2pt5 = np.array([np.nan])

    rctx = contexts.DistancesContext()
    rctx.rrup = np.array([max(1.0, float(Rrup_km))], dtype=float)

    rup = contexts.RuptureContext()
    rup.mag = np.array([float(M)], dtype=float)
    rup.ztor = np.array([ztor if ztor is not None else max(0.0, RUP_DEPTH_DEFAULT-2)], dtype=float)
    rup.dip = np.array([90.0], dtype=float)
    rup.width = np.array([np.nan])
    rup.rake = np.array([rake if rake is not None else 0.0], dtype=float)

    mean_ln, sigmas = gsim.get_mean_and_stddevs(sctx, rup, rctx, imt, [StdDev.TOTAL])
    return float(mean_ln[0]), float(sigmas[0][0])

def _pga_ln_stats_fallback(M, Rrup_km, vs30=VS30):
    """
    APPROXIMATE fallback GMPE (rock, reference Vs30~760).
    Form: ln(PGA[g]) = a + b*M - ln(R + h) - k*R
    This is NOT a substitute for a vetted GMPE; it only keeps the pipeline running.
    """
    a = -2.0      # baseline
    b = 1.10      # magnitude scaling
    h = 8.0       # pseudo-depth (km)
    k = 0.0025    # geometric+anelastic decay
    R = max(1.0, float(Rrup_km))
    mean_ln = a + b*float(M) - math.log(R + h) - k*R
    sigma = 0.6   # crude, total log std
    return mean_ln, sigma

def pga_ln_stats(M, Rrup_km, vs30=VS30, ztor=None, rake=None):
    if USE_OPENQUAKE:
        return _pga_ln_stats_openquake(M, Rrup_km, vs30, ztor, rake)
    else:
        return _pga_ln_stats_fallback(M, Rrup_km, vs30)

def prob_IM_exceeds_x(M, R_km, x_g):
    """Lognormal exceedance probability P(IM > x | M,R)."""
    if x_g <= 0:
        return 1.0
    mean_ln, sigma = pga_ln_stats(M, R_km)
    z = (math.log(x_g) - mean_ln) / sigma
    # 1 - Phi(z)
    return 0.5 * (1.0 - math.erf(z / math.sqrt(2.0)))

# ---------------- 5) HAZARD (ANNUAL RATE OF EXCEEDANCE) ----------------
# Simple annualized rate per event: 1 / years_span, then sum contributions.
EQ_LON = eq_gdf["lon"].to_numpy()
EQ_LAT = eq_gdf["lat"].to_numpy()
EQ_MAG = eq_gdf["mag_use"].to_numpy()
EQ_DEP = eq_gdf["depth_km"].to_numpy()
EVENT_RATE = 1.0 / years_span   # per-event annual rate

def annual_exceedance_rate_at_threshold(lon, lat, x_g):
    d = haversine_km(lon, lat, EQ_LON, EQ_LAT)
    mask = d <= MAX_DIST_KM
    if not np.any(mask):
        return 0.0
    # vectorized exceedance per masked events
    d_m = d[mask]
    m_m = EQ_MAG[mask]
    # (Optional: include depth -> rrup ~ sqrt(r_hyp^2 + depth^2))
    rrup = np.sqrt(d_m**2 + np.maximum(EQ_DEP[mask], 0.0)**2)
    # compute probabilities
    probs = np.array([prob_IM_exceeds_x(M, R, x_g) for M, R in zip(m_m, rrup)], dtype=float)
    lam = float(np.sum(EVENT_RATE * probs))
    return lam

def invert_for_pga_poe(lon, lat, poe=TARGET_POE, years=TARGET_YEARS):
    """Find PGA[g] such that PoE in `years` equals `poe` via bisection on lambda."""
    target_lambda = -math.log(1.0 - float(poe)) / float(years)
    # bracket (g)
    lo, hi = 1e-4, 2.0
    for _ in range(32):
        mid = math.sqrt(lo*hi)
        lam_mid = annual_exceedance_rate_at_threshold(lon, lat, mid)
        if lam_mid > target_lambda:
            hi = mid
        else:
            lo = mid
    return hi  # conservative

# ---------------- 6) COMPUTE PGA(10% in 50y) ON GRID ----------------
vals = []
tqdm_desc = f"Computing PGA (PoE={TARGET_POE*100:.0f}% in {TARGET_YEARS}y)"
for i, row in tqdm(grid.iterrows(), total=len(grid), desc=tqdm_desc):
    pga = invert_for_pga_poe(row["lon"], row["lat"], TARGET_POE, TARGET_YEARS)
    vals.append(pga)
grid["PGA_10in50_g"] = vals

# ---------------- 7) (OPTIONAL) FAULT FEATURES ----------------
# Uncomment when you have a faults shapefile and want to add features
# if os.path.exists(FAULTS):
#     faults = gpd.read_file(FAULTS).to_crs("EPSG:4326")
#     # Reproject to an equal-area/equidistant CRS for distance if you need precision.
#     # For speed, we do a crude lon/lat distance (~km) here:
#     def dist_to_fault_km(pt):
#         # quick min haversine to all fault vertices (coarse but fast)
#         # better: project to local UTM and compute exact distance
#         min_d = 1e9
#         for geom in faults.geometry:
#             if geom is None:
#                 continue
#             for x, y in getattr(geom, "coords", []):
#                 d = haversine_km(pt.x, pt.y, x, y)
#                 if d < min_d:
#                     min_d = d
#         return min_d
#     grid_gdf["dist_fault_km"] = grid_gdf.geometry.apply(dist_to_fault_km)
# else:
#     grid_gdf["dist_fault_km"] = np.nan

# ---------------- 8) TRAIN SIMPLE ML EMULATOR ----------------
# Local features around each node:
#   - rate proxy within 100km
#   - mean depth within 200km
#   - max M within 300km
def local_features(lon, lat):
    d = haversine_km(lon, lat, EQ_LON, EQ_LAT)
    f1 = float(np.mean(d <= 100.0)) / years_span
    f2 = float(np.mean(EQ_DEP[d <= 200.0])) if np.any(d <= 200.0) else RUP_DEPTH_DEFAULT
    f3 = float(np.max(EQ_MAG[d <= 300.0])) if np.any(d <= 300.0) else MIN_MAG
    return f1, f2, f3

feat_rows = [local_features(lo, la) for lo, la in zip(grid["lon"], grid["lat"])]
F = pd.DataFrame(feat_rows, columns=["rate100_ann","meanDepth200_km","maxMag300"])
F["lon"] = grid["lon"]; F["lat"] = grid["lat"]
F["PGA_10in50_g"] = grid["PGA_10in50_g"]

X = F[["rate100_ann","meanDepth200_km","maxMag300","lon","lat"]].to_numpy()
y = F["PGA_10in50_g"].to_numpy()

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBRegressor(
    n_estimators=600, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, n_jobs=-1, random_state=42
)
model.fit(Xtr, ytr)
pred = model.predict(Xte)
print(f"ML emulator R2: {r2_score(yte, pred):.3f}   MAE: {mean_absolute_error(yte, pred):.4f} g")

# Save trained model
model.save_model(os.path.join(OUT_DIR, "xgb_pga_emulator.json"))

# ---------------- 9) EXPORTS ----------------
# CSV (features + PGA)
csv_out = os.path.join(OUT_DIR, "PGA_10in50_grid.csv")
F.to_csv(csv_out, index=False)

# GeoTIFF (plate carrée, EPSG:4326)
minx, maxx = -180.0, 180.0
miny, maxy =  -90.0,  90.0
res = float(GRID_RES_DEG)

width  = int(round((maxx - minx) / res))
height = int(round((maxy - miny) / res))
arr = np.full((height, width), np.float32(np.nan))

# index helpers
lon_idx = np.clip(((F["lon"].to_numpy() - minx) / res).round().astype(int), 0, width-1)
lat_idx = np.clip(((maxy - F["lat"].to_numpy()) / res).round().astype(int), 0, height-1)

arr[lat_idx, lon_idx] = F["PGA_10in50_g"].astype("float32").to_numpy()

transform = from_origin(minx, maxy, res, res)
profile = {
    "driver":"GTiff","height":height,"width":width,"count":1,"dtype":"float32",
    "crs":"EPSG:4326","transform":transform,"compress":"lzw","nodata":np.nan
}
tif_path = os.path.join(OUT_DIR, "PGA_10in50_global.tif")
with rio.open(tif_path, "w", **profile) as dst:
    dst.write(arr, 1)

print("Wrote:", tif_path, "and", csv_out)


OpenQuake not available — using built-in approximate GMPE fallback.


Computing PGA (PoE=10% in 50y):   0%|                                           | 26/259200 [00:20<45:43:52,  1.57it/s]